In [ ]:

import matplotlib.pyplot as plt


import seaborn as sns


import pandas as pd


# EN: Load the Superstore dataset from local directory
#     Encoding ISO-8859-1 is used because the file contains special characters
# RU: Загружаем датасет Superstore из локальной директории
#     Кодировка ISO-8859-1 используется из-за наличия специальных символов
df = pd.read_csv(
    r"E:\Project_4\DATA\Sample - Superstore.csv",
    encoding="ISO-8859-1"
)


# EN: Display all column names to understand dataset structure
# RU: Выводим названия всех колонок для понимания структуры датасета
df.columns


# EN: Get general information about the dataset:
#     - number of rows
#     - data types
#     - non-null counts
#     Useful for initial data quality checks
# RU: Получаем общую информацию о датасете:
#     - количество строк
#     - типы данных
#     - количество непустых значений
#     Полезно для первичной проверки качества данных
df.info()


# EN: Generate descriptive statistics for numerical columns:
#     count, mean, std, min, max, quartiles
#     Helps to detect outliers and understand data distribution
# RU: Генерируем описательную статистику для числовых колонок:
#     количество, среднее, стандартное отклонение, минимум, максимум, квартили
#     Помогает выявить выбросы и понять распределение данных
df.describe()


In [ ]:
# EN: Standardize column names to ensure consistency and prevent KeyError issues
#     - convert all names to lowercase
#     - replace spaces with underscores
#     - replace hyphens with underscores
# RU: Стандартизируем названия колонок для единообразия и предотвращения ошибок KeyError
#     - приводим все названия к нижнему регистру
#     - заменяем пробелы на нижнее подчёркивание
#     - заменяем дефисы на нижнее подчёркивание
df.columns = [col.lower().replace(' ', '_').replace('-', '_') for col in df.columns]


# EN: Verify the corrected column names after standardization
# RU: Проверяем исправленные названия колонок после стандартизации
print("To'g'irlangan ustunlar", df.columns.tolist())


In [ ]:
# EN: Create a new figure for the plot with specified size
# RU: Создаём новое окно графика с заданным размером
plt.figure(figsize=(10, 6))


# EN: Plot a regression chart to analyze the relationship between Discount and Profit
#     - scatter points show individual observations
#     - regression line shows the overall trend
#     - alpha controls transparency to reduce overplotting
# RU: Строим регрессионный график для анализа связи между скидкой и прибылью
#     - точки показывают отдельные наблюдения
#     - линия регрессии отражает общий тренд
#     - параметр alpha уменьшает наложение точек
sns.regplot(
    data=df,
    x='discount',
    y='profit',
    scatter_kws={'alpha': 0.3},
    line_kws={'color': 'red'}
)


# EN: Add a descriptive title to the chart
# RU: Добавляем описательный заголовок графика
plt.title("Chegirma va Foyda o'rtasidagi bog'liqlik")


# EN: Label the x-axis to indicate discount values
# RU: Подписываем ось X, указывая значения скидки
plt.xlabel("chegirma (Discount)")


# EN: Label the y-axis to indicate profit values
# RU: Подписываем ось Y, указывая значения прибыли
plt.ylabel("Foyda (Profit)")


# EN: Display the final plot
# RU: Отображаем итоговый график
plt.show()


In [ ]:
# EN: Create a new figure with a larger size for better readability
# RU: Создаём новое окно графика увеличенного размера для лучшей читаемости
plt.figure(figsize=(12, 6))


# EN: Draw a boxplot to analyze profit distribution by product category
#     - x-axis: product category
#     - y-axis: profit values
#     - hue: discount levels to compare distributions within each category
#     - palette Set2 is used for clear visual distinction
#     - legend is disabled to reduce visual clutter
# RU: Строим boxplot для анализа распределения прибыли по категориям товаров
#     - ось X: категории товаров
#     - ось Y: значения прибыли
#     - hue: уровни скидок для сравнения распределений внутри категорий
#     - палитра Set2 используется для наглядного различия
#     - легенда отключена для уменьшения визуального шума
sns.boxplot(
    data=df,
    x='category',
    y='profit',
    palette='Set2',
    hue='discount',
    legend=False
)


# EN: Add a descriptive title highlighting distribution and outliers
# RU: Добавляем описательный заголовок, подчёркивающий распределение и аномалии
plt.title("Kategoriyalar bo'yicha foyda taqsimoti va anomaliyalar")


# EN: Add a grid to improve value comparison across categories
#     Dashed lines and transparency make the grid less dominant
# RU: Добавляем сетку для удобного сравнения значений между категориями
#     Пунктирные линии и прозрачность делают сетку менее навязчивой
plt.grid(True, linestyle='--', alpha=0.6)


# EN: Display the final visualization
# RU: Отображаем итоговую визуализацию
plt.show()


In [ ]:
# EN: Convert order_date column to datetime format for time-based analysis
# RU: Преобразуем колонку order_date в формат datetime для временного анализа
df['order_date'] = pd.to_datetime(df['order_date'])


# EN: Extract month from order_date for monthly aggregation
# RU: Извлекаем месяц из order_date для агрегации по месяцам
df['order_month'] = df['order_date'].dt.month


# EN: Extract year from order_date (useful for multi-year analysis)
# RU: Извлекаем год из order_date (полезно при анализе за несколько лет)
df['order_year'] = df['order_date'].dt.year


# EN: Aggregate total Sales and Profit by month
#     This helps analyze seasonal trends and performance dynamics
# RU: Агрегируем общие значения продаж и прибыли по месяцам
#     Это помогает анализировать сезонность и динамику показателей
monthly_data = (
    df.groupby('order_month')[['sales', 'profit']]
      .sum()
      .reset_index()
)


# EN: Create a line chart to visualize monthly sales and profit trends
# RU: Создаём линейный график для визуализации месячной динамики продаж и прибыли
plt.figure(figsize=(12, 6))


# EN: Plot total monthly sales
# RU: Строим линию общих месячных продаж
sns.lineplot(
    data=monthly_data,
    x='order_month',
    y='sales',
    label='Jami Savdo',
    marker='o'
)


# EN: Plot total monthly profit
# RU: Строим линию общей месячной прибыли
sns.lineplot(
    data=monthly_data,
    x='order_month',
    y='profit',
    label='Jami Foyda',
    marker='o'
)


# EN: Add chart title describing the time-based comparison
# RU: Добавляем заголовок графика, описывающий временное сравнение
plt.title("Yil davomida Savdo va Foyda dinamikasi (Oylik)")


# EN: Set x-axis ticks to represent all 12 months
# RU: Настраиваем деления оси X для отображения всех 12 месяцев
plt.xticks(range(1, 13))


# EN: Add a light grid for better trend readability
# RU: Добавляем лёгкую сетку для лучшей читаемости трендов
plt.grid(True, alpha=0.3)


# EN: Display legend to distinguish Sales and Profit lines
# RU: Отображаем легенду для различия линий продаж и прибыли
plt.legend()


# EN: Show the final plot
# RU: Отображаем итоговый график
plt.show()


# EN: Insight:
#     The chart indicates a mismatch between sales volume and profit,
#     suggesting that higher sales do not always lead to higher profitability.
#     This may be caused by excessive discounts, high costs, or unprofitable product segments.
# RU: Вывод:
#     График показывает несоответствие между объёмом продаж и прибылью,
#     что говорит о том, что рост продаж не всегда приводит к росту прибыльности.
#     Возможные причины: высокие скидки, большие издержки или убыточные сегменты товаров.


In [ ]:
# EN: Analyze the relationship between total Sales and Profit across regions
# RU: Анализируем соотношение общих продаж и прибыли по регионам
region_analysis = df.groupby('region')[['sales', 'profit']].sum().reset_index()


# EN: Calculate profit margin (%) for each region
#     Profit margin shows how much profit is generated from sales
# RU: Рассчитываем коэффициент прибыли (%) для каждого региона
#     Profit margin показывает, какую долю прибыли приносят продажи
region_analysis['profit_margin'] = (
    region_analysis['profit'] / region_analysis['sales']
) * 100


# EN: Create a bar chart to compare profit margins across regions
# RU: Создаём столбчатую диаграмму для сравнения коэффициентов прибыли по регионам
plt.figure(figsize=(10, 6))


# EN: Plot profit margin by region using a categorical bar chart
#     - viridis palette provides a clear gradient
#     - hue is set to region to emphasize category separation
# RU: Строим столбчатый график profit margin по регионам
#     - палитра viridis обеспечивает наглядный градиент
#     - параметр hue используется для визуального разделения категорий
sns.barplot(
    data=region_analysis,
    x='region',
    y='profit_margin',
    palette='viridis',
    hue='region'
)


# EN: Add a descriptive title to highlight regional performance
# RU: Добавляем описательный заголовок, подчёркивающий региональную эффективность
plt.title("Hududlar bo'yicha Foyda Koeffitsienti (%)")


# EN: Label the y-axis to indicate percentage values
# RU: Подписываем ось Y, указывая процентные значения
plt.ylabel("Profit Margin (%)")


# EN: Draw a horizontal reference line at zero
#     This helps quickly identify loss-making regions
# RU: Проводим горизонтальную линию на уровне нуля
#     Это помогает быстро определить убыточные регионы
plt.axhline(0, color='black', linewidth=1)


# EN: Display the final chart
# RU: Отображаем итоговый график
plt.show()


In [ ]:
# EN: Analyze total Profit by Sub-Category to identify which product segments are profitable or unprofitable
# RU: Анализируем общую прибыль по подкатегориям, чтобы выявить прибыльные и убыточные сегменты продуктов
sub_cat = df.groupby('sub_category')['profit'].sum().sort_values().reset_index()


# EN: Create a larger figure for better readability of sub-category names
# RU: Создаём более крупное окно графика для лучшей читаемости названий подкатегорий
plt.figure(figsize=(10, 8))


# EN: Assign colors based on profit value
#     - Red for negative profit (loss)
#     - Green for positive profit
# RU: Назначаем цвета в зависимости от значения прибыли
#     - Красный для отрицательной прибыли (убыток)
#     - Зеленый для положительной прибыли
colors = ['red' if x < 0 else 'green' for x in sub_cat['profit']]


# EN: Plot a horizontal bar chart for Sub-Category profits
#     - palette uses colors assigned above
#     - hue is set to sub_category to differentiate bars
# RU: Строим горизонтальный столбчатый график прибыли по подкатегориям
#     - палитра использует назначенные выше цвета
#     - hue установлен на sub_category для различения столбцов
sns.barplot(
    data=sub_cat,
    x='profit',
    y='sub_category',
    palette=colors,
    hue='sub_category'
)


# EN: Add a descriptive title to highlight total profit or loss by sub-category
# RU: Добавляем заголовок, подчёркивающий общую прибыль или убытки по подкатегориям
plt.title("Mahsulot kesimida jami foyda/zarar")


# EN: Add a light grid to improve readability of profit values
# RU: Добавляем лёгкую сетку для удобства чтения значений прибыли
plt.grid(True, alpha=0.3)


# EN: Display the final visualization
# RU: Отображаем итоговый график
plt.show()


In [ ]:
# EN: Calculate region-level statistics including:
#     - average discount
#     - total profit
#     - total sales
# RU: Рассчитываем статистику по регионам, включая:
#     - среднюю скидку
#     - общую прибыль
#     - общий объём продаж
region_stats = df.groupby('region').agg({
    'discount': 'mean',
    'profit': 'sum',
    'sales': 'sum'
}).reset_index()


# EN: Add profit margin (%) for each region for clarity
#     This shows the percentage of profit relative to total sales
# RU: Добавляем коэффициент прибыли (%) для каждого региона для наглядности
#     Показывает процент прибыли от общего объёма продаж
region_stats['profit_margin_%'] = (
    region_stats['profit'] / region_stats['sales']
) * 100


# EN: Print the region statistics sorted by average discount in descending order
#     Helps to quickly identify regions with highest discounts
# RU: Выводим статистику по регионам, отсортированную по средней скидке в порядке убывания
#     Это помогает быстро определить регионы с наибольшими скидками
print("Hududlar bo'yicha statistika: ")
print(region_stats.sort_values(by='discount', ascending=False))


In [ ]:
# EN: Create a scatter plot to visualize the relationship between average discount and profit margin by region
# RU: Создаём scatter plot для визуализации связи между средней скидкой и коэффициентом прибыли по регионам
plt.figure(figsize=(10, 6))


# EN: Plot each region as a point with:
#     - x-axis: average discount
#     - y-axis: profit margin (%)
#     - hue: region for color distinction
#     - s: size of the points
# RU: Строим точки для каждого региона с:
#     - ось X: средняя скидка
#     - ось Y: коэффициент прибыли (%)
#     - hue: регион для цветового различия
#     - s: размер точек
sns.scatterplot(
    data=region_stats,
    x='discount',
    y='profit_margin_%',
    hue='region',
    s=200
)


# EN: Add a descriptive title highlighting discount vs profit margin
# RU: Добавляем заголовок, подчёркивающий связь скидок и коэффициента прибыли
plt.title("Chegirma va Foyda marjasi o'rtasidagi bog'liqlik (Hududlar kesimida)")


# EN: Draw a horizontal reference line at 0% profit margin
#     Helps quickly identify regions with negative profit
# RU: Проводим горизонтальную линию на уровне 0% прибыли
#     Это помогает быстро определить регионы с отрицательной прибылью
plt.axhline(0, color='red', linestyle='--')


# EN: Add grid for better readability of data points
# RU: Добавляем сетку для удобства чтения точек на графике
plt.grid(True)


# EN: Display the final scatter plot
# RU: Отображаем итоговый scatter plot
plt.show()


In [ ]:
# EN: Filter the dataset for the 'Central' region only
# RU: Фильтруем датасет только для региона 'Central'
central_df = df[df['region'] == 'Central']


# EN: Analyze average discount and total profit by customer segment in Central region
#     - Helps identify which segments are receiving highest discounts and generating profit
# RU: Анализируем среднюю скидку и общую прибыль по сегментам клиентов в регионе Central
#     - Помогает определить, какие сегменты получают наибольшие скидки и приносят прибыль
segment_analysis = (
    central_df.groupby('segment')
    .agg({
        'discount': 'mean',
        'profit': 'sum'
    })
    .sort_values(by='discount', ascending=False)
    .reset_index()
)


# EN: Print the segment-level analysis for Central region
# RU: Выводим анализ по сегментам для региона Central
print("Markaziy hududda segmentlar tahlili :")
print(segment_analysis)


In [ ]:
# EN: Analyze average Profit, Sales, and Discount by Ship Mode
#     - Helps understand which delivery method is most profitable
# RU: Анализируем среднюю прибыль, продажи и скидки по типу доставки
#     - Помогает понять, какой способ доставки наиболее прибыльный
ship_analysis = df.groupby('ship_mode').agg({
    'profit': 'mean',
    'sales': 'mean',
    'discount': 'mean'
}).reset_index()


# EN: Create a figure for the bar plot with appropriate size
# RU: Создаём окно графика подходящего размера для столбчатой диаграммы
plt.figure(figsize=(10, 6))


# EN: Plot average profit by Ship Mode
#     - palette 'magma' gives a visually appealing gradient
#     - hue is set to ship_mode to differentiate categories
# RU: Строим среднюю прибыль по типу доставки
#     - палитра 'magma' обеспечивает наглядный градиент
#     - hue установлен на ship_mode для различия категорий
sns.barplot(
    data=ship_analysis,
    x='ship_mode',
    y='profit',
    palette='magma',
    hue='ship_mode'
)


# EN: Add title describing average profit by delivery method
# RU: Добавляем заголовок, описывающий среднюю прибыль по типу доставки
plt.title("Yetkazib berish turi bo'yicha o'rtacha foyda")


# EN: Label y-axis to indicate average profit
# RU: Подписываем ось Y, указывая среднюю прибыль
plt.ylabel("O'rtacha Foyda (Average Profit)")


# EN: Display the final bar chart
# RU: Отображаем итоговую столбчатую диаграмму
plt.show()


In [ ]:
# EN: Calculate profit margin (%) for each row
#     This new column will be useful for Power BI dashboards and further analysis
# RU: Рассчитываем коэффициент прибыли (%) для каждой строки
#     Этот новый столбец будет полезен для дашбордов Power BI и дальнейшего анализа
df['profit_margin'] = (df['profit'] / df['sales']) * 100


# EN: Save the updated dataset with all new columns (month, year, profit margin) to a CSV file
#     - index=False ensures row numbers are not saved as a separate column
# RU: Сохраняем обновлённый датасет со всеми новыми колонками (месяц, год, коэффициент прибыли) в CSV файл
#     - index=False предотвращает сохранение номеров строк как отдельного столбца
df.to_csv(r"E:\Project_4\DATA\Ready - Superstore.csv", index=False)


# EN: Print confirmation message after saving the file
# RU: Выводим сообщение о подтверждении после сохранения файла
print('File saqlandi')
